In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.simplefilter('ignore')


In [30]:
# データの読み込み
df_train_org = pd.read_csv('./data/train.csv')
df_test_org = pd.read_csv('./data/test.csv')
# df = pd.read_csv('../data/sig740/train.csv')


In [56]:
# # 成功（successful）と失敗（failed）以外のデータを除外する
# df = df_org[(df_org['state'] == 'successful') | (df_org['state'] == 'failed')]

In [24]:
df_train = df_train_org
df_test = df_test_org

In [5]:
df_train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,Travel_Rarely,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,Travel_Rarely,706.247579,Research & Development,14.791373,1,Medical,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,Travel_Rarely,500.610860,Research & Development,2.146966,4,Medical,981,2,...,80,1,8,3,3,1,0,0,0,0


In [7]:
df_train.columns

Index(['id', 'Age', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'NumCompaniesWorked', 'Over18', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StandardHours', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Attrition'],
      dtype='object')

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        1200 non-null   int64  
 1   Age                       1200 non-null   int64  
 2   BusinessTravel            1200 non-null   object 
 3   DailyRate                 1200 non-null   float64
 4   Department                1200 non-null   object 
 5   DistanceFromHome          1200 non-null   float64
 6   Education                 1200 non-null   int64  
 7   EducationField            1200 non-null   object 
 8   EmployeeNumber            1200 non-null   int64  
 9   EnvironmentSatisfaction   1200 non-null   int64  
 10  Gender                    1200 non-null   object 
 11  HourlyRate                1200 non-null   int64  
 12  JobInvolvement            1200 non-null   int64  
 13  JobLevel                  1200 non-null   int64  
 14  JobRole 

## プロジェクトの状態（state）

In [9]:
# state_mapping = { 'successful':1, 'failed':0}
# df['state'] = df['state'].map(state_mapping)

## 日時

### 期間を新しく作成する
deadline - launched

In [10]:
# # 日時に関する列を string から TimeStamp に変換
# df['deadline'] = pd.to_datetime(df['deadline'])
# df['launched'] = pd.to_datetime(df['launched'])

# # 期間の列を作成
# df['period'] = (df['deadline'] - df['launched']).dt.days # 差分を日数として表示

### launched
周期性を考慮してクォーターで分ける
米国の会計年度は10月から9月。

In [11]:
# df['launched_year'] = df['launched'].dt.year
# df['launched_month'] = df['launched'].dt.month
# df['launched_day'] = df['launched'].dt.day

### 不要な列を削除する

In [12]:
# # launched と deadline を削除
# df = df.drop(columns=['deadline', 'launched'])

## カテゴリ

In [13]:
# # main_categoryとcategoryを結合する（class）
# df['class'] = df['main_category'].str.cat(df['category'], sep='**')
# df.head()

In [14]:
# # 各classの個数をカウントし、dataframeに変換する
# # https://www.web-dev-qa-db-ja.com/ja/python/python-pandas%EF%BC%9A-valuecounts%E5%87%BA%E5%8A%9B%E3%82%92%E3%83%87%E3%83%BC%E3%82%BF%E3%83%95%E3%83%AC%E3%83%BC%E3%83%A0%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%97%E3%81%BE%E3%81%99/835447736/
# df_counts = df['class'].value_counts().rename_axis('class').reset_index(name='counts')
# # トップ40位を抜き出す
# df_largest = df_counts.nlargest(120, columns='counts')
# # トップ40位の要素を削除したdataframeを作成する
# df_others = df_counts.drop(df_largest.index)
# # df_othersのclass列の項目をリスト化する
# others_list = df_others['class'].to_list()
# # others_list内の文字列に一致するclassをOthersに変更する
# df = df.replace(others_list, 'Others')

In [16]:
# df.head()

## プロジェクト名
nameをどうにかする

### 単語数を抽出

In [17]:
# # 単語数をカウントする関数
# def count_words(sentence):
#     words_list = str(sentence).split(' ') 
#     return len(words_list)
# # 単語数をカウントする関数を name の各データに適用
# df['n_words'] = df['name'].apply(count_words) 
# # name を削除
# df = df.drop(columns=['name'])

## 目標金額
usd_goal_realを対数変換する

In [18]:
# df['usd_goal_real'].head()

In [19]:
# df['usd_goal_real'].plot(bins=20, kind='hist')

In [20]:
# # usd_goal_realを対数変換する
# df['log_goal'] = np.log(df['usd_goal_real'])
# df = df.drop(columns='usd_goal_real')

# 加工データの保存

In [27]:
# df.to_csv('./data/input_01.csv')
# df.to_csv('./data/input_02.csv')
# profile.to_file("./data/sig740/KS_ProfileReport.html")
df_train.to_csv('../data/sig740/preprocessed_train.csv')
df_test.to_csv('../data/sig740/preprocessed_test.csv')

### input_01.csv
- 初期

### input_02.csv
- input_01
- main_category x category -> class

### input_03.csv
- input_01
- main_category x category -> class
- launched -> year/month/day

### input_04.csv・・・未作成（name除去だけなら3_学習でやればよい）
- input_03
- nameを除去